In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
from my_functions import replace

In [3]:
df = pd.DataFrame()
for century in range(1, 11):
    df2 = pd.DataFrame()
    df2['prophecy'] = pd.read_csv(f'./nostradamus_docs/nostradamus_century_{century}.txt', sep = '\t', header = None)[0]
    df2['century'] = century
    df = pd.concat([df, df2], axis = 0, sort = False)    

In [4]:
df.head()

,prophecy,century
0,Century I,1
1,1,1
2,Sitting alone at night in secret study;,1
3,it is placed on the brass tripod.,1
4,A slight flame comes out of the emptiness and,1


In [5]:
df = df[(['Century' not in s for s in df['prophecy']])].reset_index()
df.drop('index', axis = 1, inplace = True)

In [6]:
df = df[np.mod(np.arange(df.index.size), 5) != 0].reset_index()
df.drop('index', axis = 1, inplace = True)

In [7]:
df.loc[df.index % 4 == 0, 'line_num'] = 1
df.loc[df.index % 4 == 1, 'line_num'] = 2
df.loc[df.index % 4 == 2, 'line_num'] = 3
df.loc[df.index % 4 == 3, 'line_num'] = 4

In [8]:
n = 0
century = 1
boolean_list = (df.index % 4 == 0).tolist()
for i, boolean in enumerate(boolean_list):
    if boolean and df.loc[i, 'century'] == century:
        n+=1
        df.loc[i, 'prophecy_num'] = n
    elif boolean:
        century+=1
        n = 1
        df.loc[i, 'prophecy_num'] = n
    else:
        df.loc[i, 'prophecy_num'] = n

In [9]:
df = df.groupby(['century', 'prophecy_num'])['prophecy'].apply(lambda x: '  '.join(x)).reset_index()

In [10]:
df['century'] = df['century'].astype(float)

In [11]:
len(df)

942

In [12]:
df.head()

,century,prophecy_num,prophecy
0,1.0,1.0,Sitting alone at night in secret study; it is...
1,1.0,2.0,The wand in the hand is placed in the middle o...
2,1.0,3.0,When the litters are overturned by the whirlwi...
3,1.0,4.0,In the world there will be made a king who wi...
4,1.0,5.0,They will be driven away for a long drawn out ...


In [13]:
h2_df = pd.read_csv('./nostradamus_docs/nostradamus_epistle_henry_2.txt', sep = '\t', header = None)
h2_df['group'] = 1
h2_df = h2_df.loc[2:].groupby(['group'])[0].apply(lambda x: '  '.join(x)).reset_index()

In [14]:
pre_df = pd.read_csv('./nostradamus_docs/nostradamus_preface.txt', sep = '\t', header = None)
pre_df['group'] = 1
pre_df = pre_df.loc[2:].groupby(['group'])[0].apply(lambda x: '  '.join(x)).reset_index()

In [15]:
from pymongo import MongoClient

In [18]:
client = MongoClient()
db = client.nostradamus
db.create_collection('prophecy')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nostradamus'), 'prophecy')

In [19]:
# db.drop_collection('prophecy')

In [20]:
client.address

('localhost', 27017)

In [21]:
client.list_database_names()

['MyDatabase', 'admin', 'config', 'local', 'nostradamus']

In [22]:
db.list_collection_names()

['prophecy']

In [23]:
col = db.get_collection('prophecy')

In [24]:
col.estimated_document_count()

0

In [25]:
for i, prophecy in enumerate(df['prophecy']):
    doc = {'prophecy': prophecy, 'century': df['century'][i], 'prophecy_num': df['prophecy_num'][i]}
    col.insert_one(doc)

In [26]:
doc = {'epistle_henry_ii': pd.read_csv('./nostradamus_docs/nostradamus_epistle_henry_2.txt', sep = '\t', header = None)[0]}
col.insert_one(doc)

InvalidDocument: Cannot encode object: 0                                   Epistle to Henry II
1                                   EPISTLE TO HENRY II
2     TO THE MOST INVINCIBLE MOST POWERFUL AND MOST ...
3     Ever since my long-beclouded face first presen...
4     While I was seized with this singular desire t...
5     It is to a most prudent and most wise Prince t...
6     Indeed, someone, who would do well to blow his...
7     Nevertheless, I wanted to leave a record in wr...
8     I have calculated and composed all during choi...
9     It is quite true, Sire, that my natural instin...
10    There are some who would attribute to me that ...
11    Notwithstanding those who cannot contain the m...
12    Such is the extent of time past, subject to co...
13    Although my calculations may not hold good for...
14    As for myself, I would never claim such a titl...
15    For God will take notice of the long barrennes...
16    There will be great differences between the th...
17    The

In [ ]:
doc = {'preface': pd.read_csv('./nostradamus_docs/nostradamus_preface.txt', sep = '\t', header = None)[0]}
col.insert_one(doc)

In [ ]:
col.estimated_document_count()

In [ ]:
# for doc in col.find():
#     print(doc)

In [27]:
df = pd.DataFrame(list(col.find()))
df.drop('_id', axis = 1, inplace = True)

In [28]:
substitutions = {'african': 'africa', 'barbaric': 'barbarian', 'blood': 'bleed', 'bloody': 'bleed',
                 'bled': 'bleed', 'burnt': 'burn', 'captive': 'capture', 'captive': 'captured',
                 'children': 'child', 'complaints': 'complain', 'dead': 'die', 'death': 'die',
                 'deceive': 'deceit', 'depth': 'deep', 'defense': 'defend', 'destruction': 'destroy',
                 'discovered': 'discover', 'discoveries': 'discover', 'discovery': 'discover',
                 'doubly': 'double', 'drawn': 'draw', 'dreamer': 'dream', 'driven': 'drive',
                 'drunken': 'drunk', 'eastern': 'east', 'eaten': 'eat', 'enclosed': 'enclose',
                 'enclosure': 'enclose', 'entry': 'enter', 'envied': 'envy', 'envious': 'envy',
                 'executioners': 'execute', 'extension': 'extend', 'falsify': 'false', 'failure': 'fail',
                 'flight': 'flies', 'florense': 'florence', 'french': 'france', 'fraudulent': 'fraud',
                 'friendship': 'friend', 'frightful': 'frighten', 'frost': 'freeze', 'frozen': 'freeze',
                 'furious': 'fury', 'gascony': 'gascon', 'genevans': 'geneva', 'gnashing': 'gnaw',
                 'governor': 'govern', 'greedy': 'greed', 'germany': 'german', 'golden': 'gold',
                 'greatest': 'great', 'greater': 'great', 'halfway': 'half', 'healthy': 'health',
                 'hairy': 'hair', 'hardships': 'hard', 'hatred': 'hate', 'heard': 'hear', 'higher': 'high',
                 'highest': 'high', 'household': 'house', 'hungarians': 'hungary', 'impetuosity': 'impetuous',
                 'injured': 'injure', 'italian': 'italy', 'judges': 'judge', 'judgement': 'judge',
                 'kingdom': 'king', 'larger': 'large', 'later': 'late', 'leadership': 'lead', 'leader': 'lead',
                 'leaderless': 'lead', 'leaguers': 'league', 'longer': 'long', 'loss': 'lose',
                 'lost': 'lose', 'lower': 'low', 'lowest': 'low', 'lover': 'love', 'lusitanian': 'lusitania',
                 'macedonian': 'macedonia', 'mankind': 'man', 'marriage': 'marry', 'maritime': 'marine',
                 'mightily': 'mighty', 'newly': 'new', 'older': 'old', 'oldest': 'old', 'oppose': 'opposite',
                 'pleasing': 'pleasure', 'philosophers': 'philosophy', 'provincial': 'province',
                 'pursuers': 'pursue', 'pursuit': 'pursue', 'robbery': 'rob', 'robber': 'rob',
                 'redden': 'red', 'redbeard': 'red', 'remainder': 'remain', 'revolutions': 'revolt',
                 'robbed': 'rob', 'shadow': 'shade', 'seventh': 'seven', 'shorter': 'short',
                 'smaller': 'small', 'soldiery': 'soldiers', 'sovereignty': 'sovereign', 'spacious': 'space',
                 'spanish': 'spain', 'spaniards': 'spain', 'strongest': 'strong', 'stronghold': 'strong',
                 'survivor': 'survive', 'swampy': 'swamps', 'sprung': 'spring', 'taken': 'take',
                 'taker': 'take', 'terrified': 'terror', 'terrible': 'terror', 'thirsty': 'thirst',
                 'thought': 'think', 'threatens': 'threat', 'toulousans': 'toulouse', 'thunderbolt': 'thunder',
                 'treacherous': 'treachery', 'trickery': 'trick', 'tuscany': 'tuscan', 'twentieth': 'twenty',
                 'tyrant': 'tyranny', 'victorious': 'victor', 'victory': 'victor', 'violent': 'violence',
                 'younger': 'young', 'warlike': 'war', 'weak': 'weaken', 'weaker': 'weaken', 'wives': 'wife',
                 'wooden': 'wood', 'worldwide': 'world', 'worthy': 'worth', 'youth': 'young'}

df['prophecy'] = deepcopy(replace(df['prophecy'], substitutions))
h2_df[0] = deepcopy(replace(h2_df[0], substitutions))
pre_df[0] = deepcopy(replace(pre_df[0], substitutions))

In [29]:
df.head()

,century,prophecy,prophecy_num
0,1.0,Sitting alone at night in secret study; it is...,1.0
1,1.0,The wand in the hand is placed in the middle o...,2.0
2,1.0,When the litters are overturned by the whirlwi...,3.0
3,1.0,In the world there will be made a king who wi...,4.0
4,1.0,They will be drive away for a long draw out fi...,5.0


In [30]:
h2_df

,group,0
0,1,TO THE MOST INVINCIBLE MOST POWERFUL AND MOST ...


In [31]:
df.to_pickle('prophecy_df.pkl')
h2_df.to_pickle('h2_df.pkl')
pre_df.to_pickle('pre_df.pkl')